# Predicting Player's Data Contibutions Based on their Experience #

*Skye Fort, Lauren Friesen, Taylor Jackson & Sam Thompson*

## Introduction ##
Minecraft, a game of blocks, building, and exploration developed in Java, has captivated millions worldwide. Over the years, its interactive nature has made it a platform of growing interest for research, particularly with advancements in AI. UBC PLAI Lab attempts to train a generative Ai model using audio recording of multiplayer minecraft. This project aims to create accurate predictive tools for targeted demographic research.

The central question is: How effectively can self-reported experience and age predict a player's contribution to the study, as measured by playtime hours? To address this, we will utilize the Plaicraft dataset, which provides detailed player data, including exact playtime, age, self-reported experience, and derived metrics like total sessions played.

By analyzing these variables, this project seeks to identify the most significant factors and relationships that define key player demographics. The ultimate goal is to help researchers pinpoint and engage suitable participants for future studies effectively.


## Methods ##

In [3]:
#Load R library packages to access all necessary functions
library(tidyverse)
library(repr)
library(rvest)
library(tidymodels)
library(dplyr)
library(ggplot2)


Attaching package: ‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding


── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Learn how to get started at https://www.tidymodels.org/start/



In [2]:
#read data from url
players_url <- "https://drive.google.com/uc?export=download&id=1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz"
sessions_url <- "https://drive.google.com/uc?export=download&id=14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB"

players_data<-read_csv(players_url)
head(players_data)

sessions_data<-read_csv(sessions_url)
head(sessions_data)

Rows: 196 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, age
lgl (3): subscribe, individualId, organizationName

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience subscribe
1 Pro         TRUE    
2 Veteran     TRUE    
3 Veteran    FALSE    
4 Amateur     TRUE    
5 Regular     TRUE    
6 Amateur     TRUE    
  hashedEmail                                                      played_hours
1 f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d 30.3        
2 f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9  3.8        
3 b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28  0.0        
4 23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5  0.7        
5 7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e  0.1        
6 f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977  0.0        
  name      gender age individualId organizationName
1 Morgan    Male    9  NA           NA              
2 Christian Male   17  NA           NA              
3 Blake     Male   17  NA           NA              
4 Flora     Female 21  NA           NA              
5 Kylie     Male   21  NA           NA              
6 Adrian    Female 17  NA           NA

Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


hashedEmail                                                     
1 bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf
2 36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686
3 f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc
4 bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf
5 36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686
6 bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf
  start_time       end_time         original_start_time original_end_time
1 30/06/2024 18:12 30/06/2024 18:24 1.71977e+12         1.71977e+12      
2 17/06/2024 23:33 17/06/2024 23:46 1.71867e+12         1.71867e+12      
3 25/07/2024 17:34 25/07/2024 17:57 1.72193e+12         1.72193e+12      
4 25/07/2024 03:22 25/07/2024 03:58 1.72188e+12         1.72188e+12      
5 25/05/2024 16:01 25/05/2024 16:12 1.71665e+12         1.71665e+12      
6 23/06/2024 15:08 23/06/2024 17:10 1.71916e+12         1.71916e+12

In [5]:
contribution <- players_data |>
    select(experience, played_hours, age)
head(contribution)

experience played_hours age
1 Pro        30.3          9 
2 Veteran     3.8         17 
3 Veteran     0.0         17 
4 Amateur     0.7         21 
5 Regular     0.1         21 
6 Amateur     0.0         17

In [7]:
set.seed(2000)

contribution_split <- initial_split(contribution, prop = 0.6, strata = played_hours) 

contribution_training <- training(contribution_split)
contribution_testing <- testing(contribution_split) 

In [8]:
options(repr.plot.height = 8, repr.plot.width = 9)
contribution_pairplot <- contribution_training |>
    ggpairs(mapping = aes(alpha = 0.4)) +
    theme(text = element_text(size = 20))
contribution_pairplot

ERROR: Error in ggpairs(contribution_training, mapping = aes(alpha = 0.4)): could not find function "ggpairs"
